# Supply Chain Data Preprocessing & Analysis

In [91]:
# 1. Initial Setup and Data Loading
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import StandardScaler, RobustScaler
from scipy import stats

# Load dataset
file_path = "../Data/dynamic_supply_chain_logistics_dataset.csv"
df = pd.read_csv(file_path)

# Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')

# Display initial information
print("Dataset Shape:", df.shape)
print("\nColumn Names:")
print(df.columns.tolist())
print("\nData Types:")
df.info()

Dataset Shape: (32065, 26)

Column Names:
['timestamp', 'vehicle_gps_latitude', 'vehicle_gps_longitude', 'fuel_consumption_rate', 'eta_variation_hours', 'traffic_congestion_level', 'warehouse_inventory_level', 'loading_unloading_time', 'handling_equipment_availability', 'order_fulfillment_status', 'weather_condition_severity', 'port_congestion_level', 'shipping_costs', 'supplier_reliability_score', 'lead_time_days', 'historical_demand', 'iot_temperature', 'cargo_condition_status', 'route_risk_level', 'customs_clearance_time', 'driver_behavior_score', 'fatigue_monitoring_score', 'disruption_likelihood_score', 'delay_probability', 'risk_classification', 'delivery_time_deviation']

Data Types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32065 entries, 0 to 32064
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   timestamp                        32065 non-null  object 
 1   

### Initial Data Analysis Results

The dataset contains supply chain logistics data with the following characteristics:
- **Size**: 32,065 records with 26 features
- **Features**: Mix of numerical and categorical variables
- **Time Range**: Spans multiple time periods
- **Key Metrics**: Includes various logistics and performance indicators

**Key Features**:
- Temporal: timestamp
- Location: vehicle_gps_latitude, vehicle_gps_longitude
- Performance: delivery_time_deviation, eta_variation_hours
- Risk: disruption_likelihood_score, route_risk_level
- Operations: warehouse_inventory_level, loading_unloading_time

In [92]:
# 2. Data Cleaning and Type Conversion
# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Check for missing values and duplicates
missing_values = df.isnull().sum()
missing_percentages = (missing_values / len(df)) * 100
duplicates = df.duplicated().sum()

print('Missing Values Analysis:')
print(missing_percentages[missing_percentages > 0])
print(f'\nDuplicate rows: {duplicates}')

# Display basic statistics
print('\nNumerical Columns Statistics:')
print(df.describe())

Missing Values Analysis:
Series([], dtype: float64)

Duplicate rows: 0

Numerical Columns Statistics:
                           timestamp  vehicle_gps_latitude  \
count                          32065             32065.000   
mean   2022-10-31 00:00:00.000000256                38.024   
min              2021-01-01 00:00:00                30.000   
25%              2021-12-01 00:00:00                31.281   
50%              2022-10-31 00:00:00                36.414   
75%              2023-09-30 00:00:00                44.454   
max              2024-08-29 00:00:00                50.000   
std                              NaN                 6.918   

       vehicle_gps_longitude  fuel_consumption_rate  eta_variation_hours  \
count              32065.000              32065.000            32065.000   
mean                 -90.117                  8.012                2.893   
min                 -120.000                  5.000               -2.000   
25%                 -106.254       

### Data Cleaning Results
- Timestamp conversion completed successfully
- No missing values identified in any columns
- No duplicate records found
- Numerical columns show reasonable value distributions

In [93]:
# 3. Feature Engineering
# Create derived features
df['date'] = df['timestamp'].dt.date
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.day_name()
df['month'] = df['timestamp'].dt.month

# Calculate total risk score (combining multiple risk factors)
df['total_risk_score'] = (df['disruption_likelihood_score'] + 
                        df['route_risk_level'] + 
                        df['delay_probability']) / 3

# Calculate delivery efficiency score
df['delivery_efficiency'] = 100 - (abs(df['delivery_time_deviation']) * 10 + 
                                 abs(df['eta_variation_hours']) * 5)

# Categorize delays
df['delay_category'] = pd.cut(df['delivery_time_deviation'],
                            bins=[-float('inf'), -2, -0.5, 0.5, 2, float('inf')],
                            labels=['Very Early', 'Early', 'On Time', 'Late', 'Very Late'])

# Clean and normalize scores to 0-100 scale where needed
cols_to_normalize = ['driver_behavior_score', 'fatigue_monitoring_score', 
                    'supplier_reliability_score']

for col in cols_to_normalize:
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min()) * 100

# Calculate distance between consecutive points with improved handling
df['distance'] = np.sqrt(
    (df['vehicle_gps_latitude'].diff())**2 + 
    (df['vehicle_gps_longitude'].diff())**2
)

# Fill first row's distance with the mean of next 5 rows and round all distances
first_rows_mean = df['distance'][1:6].mean()
df['distance'] = df['distance'].fillna(first_rows_mean).round(3)

print("Data preprocessing completed. New features created:")
print("\nShape of dataset:", df.shape)
print("\nNew columns added:", 
      [col for col in df.columns if col not in ['timestamp', 'vehicle_gps_latitude', 'vehicle_gps_longitude', 
       'fuel_consumption_rate', 'eta_variation_hours', 'traffic_congestion_level', 
       'warehouse_inventory_level', 'loading_unloading_time', 'handling_equipment_availability',
       'order_fulfillment_status', 'weather_condition_severity', 'port_congestion_level',
       'shipping_costs', 'supplier_reliability_score', 'lead_time_days', 'historical_demand',
       'iot_temperature', 'cargo_condition_status', 'route_risk_level', 'customs_clearance_time',
       'driver_behavior_score', 'fatigue_monitoring_score', 'disruption_likelihood_score',
       'delay_probability', 'risk_classification', 'delivery_time_deviation']])

Data preprocessing completed. New features created:

Shape of dataset: (32065, 34)

New columns added: ['date', 'hour', 'day_of_week', 'month', 'total_risk_score', 'delivery_efficiency', 'delay_category', 'distance']


### Feature Engineering Results
- Created temporal features: date, hour, day_of_week, month
- Added composite metrics: total_risk_score, delivery_efficiency
- Created delay categories for better analysis
- Normalized relevant scores to 0-100 scale
- Added distance calculations between consecutive points

In [94]:
# 4. Performance Metrics Creation
# Add delivery performance score
df['delivery_performance'] = df.apply(lambda x: 
    100 * (1 - abs(x['delivery_time_deviation'])/10) * 
    (1 - x['delay_probability']) * 
    (1 - x['disruption_likelihood_score']), axis=1)

# Add cost efficiency metric
df['cost_efficiency'] = df['shipping_costs'].mean() / df['shipping_costs'] * 100

### Performance Metrics Results
- Created delivery_performance score combining multiple factors
- Added cost_efficiency metric based on shipping costs
- New metrics provide better insights into supply chain performance

In [95]:
# 5. Outlier Detection and Handling
# Handle outliers using IQR method
def handle_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = df[column].clip(lower_bound, upper_bound)

numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    handle_outliers(df, col)

# Handle decimals for numeric columns
# Add after the outlier handling section and before final validation

# Round numeric measurements and scores to appropriate decimal places
decimal_handling = {
    # GPS coordinates (higher precision needed)
    'vehicle_gps_latitude': 6,
    'vehicle_gps_longitude': 6,
    # Distance and time measurements
    'distance': 3,
    'eta_variation_hours': 2,
    'lead_time_days': 1,
    'loading_unloading_time': 2,
    'customs_clearance_time': 2,
    # Scores and metrics (2 decimals for percentage-based metrics)
    'driver_behavior_score': 2,
    'fatigue_monitoring_score': 2,
    'supplier_reliability_score': 2,
    'delivery_efficiency': 2,
    'total_risk_score': 2,
    'cost_efficiency': 2,
    'delivery_performance': 2,
    # Other measurements
    'fuel_consumption_rate': 2,
    'shipping_costs': 2,
    'iot_temperature': 1,
    # Probabilities and indices (3 decimals for more precise probability values)
    'delay_probability': 3,
    'disruption_likelihood_score': 3
}

# Apply rounding to specified columns
for col, decimals in decimal_handling.items():
    if col in df.columns:
        df[col] = df[col].round(decimals)

print("Decimal handling completed for numeric columns")

Decimal handling completed for numeric columns


### Outlier Handling Results
- Applied IQR method to handle outliers
- Outliers capped at 1.5 * IQR boundaries
- Maintained data integrity while removing extreme values
- Treated all numeric columns for consistency

In [96]:
# 6. Final Validation and Export
# Add data validation checks
validation_results = {
    'timestamp_range': df['timestamp'].min() >= pd.Timestamp('2021-01-01'),
    'gps_coords_valid': all([-90 <= df['vehicle_gps_latitude'].max() <= 90,
                            -180 <= df['vehicle_gps_longitude'].max() <= 180]),
    'scores_normalized': all(df[cols_to_normalize].max() <= 100)
}

print('\nValidation Results:')
for check, result in validation_results.items():
    print(f'{check}: {"Passed" if result else "Failed"}')

# Save the cleaned dataset
cleaned_file_path = "../Data/cleaned_supply_chain_logistics_dataset.csv"
df.to_csv(cleaned_file_path, index=False)

print(f"Cleaned dataset saved to {cleaned_file_path}")


Validation Results:
timestamp_range: Passed
gps_coords_valid: Passed
scores_normalized: Passed
Cleaned dataset saved to ../Data/cleaned_supply_chain_logistics_dataset.csv
Cleaned dataset saved to ../Data/cleaned_supply_chain_logistics_dataset.csv


In [97]:
df.head()

,timestamp,vehicle_gps_latitude,vehicle_gps_longitude,fuel_consumption_rate,eta_variation_hours,traffic_congestion_level,warehouse_inventory_level,loading_unloading_time,handling_equipment_availability,order_fulfillment_status,weather_condition_severity,port_congestion_level,shipping_costs,supplier_reliability_score,lead_time_days,historical_demand,iot_temperature,cargo_condition_status,route_risk_level,customs_clearance_time,driver_behavior_score,fatigue_monitoring_score,disruption_likelihood_score,delay_probability,risk_classification,delivery_time_deviation,date,hour,day_of_week,month,total_risk_score,delivery_efficiency,delay_category,distance,delivery_performance,cost_efficiency
0,2021-01-01 00:00:00,40.376,-77.014,5.140,5.000,5.928,985.717,4.950,0.481,0.761,0.359,4.289,456.500,98.610,2.100,100.773,0.600,0.777,1.182,0.500,3.380,97.860,0.506,0.885,Moderate Risk,9.111,2021-01-01,0,Friday,1,0.860,-16.100,Very Late,30.121,0.500,100.630
1,2021-01-01 01:00:00,33.508,-117.037,5.100,0.980,1.592,396.700,1.030,0.621,0.197,0.231,7.018,640.410,46.320,12.600,5313.738,-9.800,0.092,9.612,0.970,20.170,91.860,0.981,0.544,High Risk,8.175,2021-01-01,1,Friday,1,3.710,13.320,Very Late,40.608,0.160,71.730
2,2021-01-01 02:00:00,30.021,-75.269,5.090,4.970,8.788,832.409,4.220,0.811,0.153,0.027,9.605,155.750,59.860,14.100,1595.049,-6.500,0.254,6.570,0.950,26.400,39.420,0.999,0.803,High Risk,1.284,2021-01-01,2,Friday,1,2.790,62.300,Late,41.913,0.020,294.940
3,2021-01-01 03:00:00,36.649,-70.191,8.220,3.100,0.045,0.573,0.530,0.009,0.812,0.617,8.900,104.320,74.270,3.800,2530.432,-0.200,0.878,0.549,4.670,36.290,90.540,0.993,0.026,High Risk,9.305,2021-01-01,3,Friday,1,0.520,-8.520,Very Late,8.351,0.050,440.360
4,2021-01-01 04:00:00,30.001,-70.012,5.000,3.220,8.005,914.925,3.620,0.020,0.054,0.952,0.062,977.220,43.110,14.000,2146.190,2.400,0.262,8.861,3.450,1.700,25.870,0.912,0.991,High Risk,7.752,2021-01-01,4,Friday,1,3.590,6.390,Very Late,6.650,0.020,47.010
